In [ ]:
import sys
import snap
import itertools
from IPython.display import Image, display

print("Hello World")

In [ ]:
status = False
try:
    import snap
    version = snap.Version
    i = snap.TInt(5)
    if i == 5:
        status = True
except:
    pass

if status:
    print("SUCCESS, your version of Snap.py is %s" % (version))
else:
    print("*** ERROR, no working Snap.py was found on your computer")


## Read .txt

In [ ]:
FOLDER_NAME = "./"
case_name = "stackoverflow-Java"
case_txt = FOLDER_NAME + case_name+".txt"
case_png = FOLDER_NAME + case_name+".png"
G5 = snap.LoadEdgeList(snap.PNGraph, case_txt, 0, 1)

In [ ]:
def get_nodes_and_edges(my_graph):
    n_nodes = my_graph.GetNodes()
    n_edges = my_graph.GetEdges()
    print(f"The graph has {n_nodes:,d} nodes; {n_edges:,d} edges")

In [ ]:
get_nodes_and_edges(G5)

## Weakly connected component

### Q1

In [ ]:
## TCnComV, a vector of connected components;
wcc = snap.TCnComV()
snap.GetWccs(G5, wcc)
print(f"Weakly connected components: {len(list(wcc)):,d}")

### (optional) Q1

In [ ]:
for CnCom in list(wcc)[:5]:
    print("Size of component: %d" % CnCom.Len())

In [ ]:
ComponentDist = snap.TIntPrV()
snap.GetWccSzCnt(G5, ComponentDist)
for comp in ComponentDist:
    print("Size: %d - Number of Components: %d" % (comp.GetVal1(), comp.GetVal2()))

In [ ]:
node_id = []

for CnCom in list(wcc)[1:2]:
    print("Size of component: %d" % CnCom.Len())
    for NI in CnCom:
        node_id.append(NI)

print(node_id)

In [ ]:
def func_in_out_degree(my_graph, node_id):
    InDegV = snap.TIntPrV()
    snap.GetNodeInDegV(my_graph, InDegV)

    OutDegV = snap.TIntPrV()
    snap.GetNodeOutDegV(my_graph, OutDegV)

    print("--- In Degree ---")
    for item in InDegV:
        if item.GetVal1() in node_id:
            print("node ID %d: in-degree %d" % (item.GetVal1(), item.GetVal2()))

    print("--- Out Degree ---")
    for item in OutDegV:
        if item.GetVal1() in node_id:
            print("node ID %d: out-degree %d" % (item.GetVal1(), item.GetVal2()))

In [ ]:
func_in_out_degree(G5, node_id)

In [ ]:
def get_subgraph(my_large_graph, node_id):
    NIdV = snap.TIntV()
    for i in node_id:
        NIdV.Add(i)

    SubGraph = snap.GetSubGraph(my_large_graph, NIdV)
    for EI in SubGraph.Edges():
        print("edge (%d %d)" % (EI.GetSrcNId(), EI.GetDstNId()))
    return SubGraph

In [ ]:
sub_graph = get_subgraph(G5, node_id)

In [ ]:
case_name = "SubGraph"
case_txt = case_name+".txt"
case_png = case_name+".png"

snap.DrawGViz(sub_graph, snap.gvlDot, case_png, case_name)
Image(case_png)

### Q2

In [ ]:
## get largest weakly connected component of G
wccLargestG = snap.GetMxWcc(G5)

# Define a vector of pairs of integers (size, count) and 
# get a distribution of connected components (component size, count):

ComponentDist = snap.TIntPrV()
snap.GetWccSzCnt(wccLargestG, ComponentDist)
for comp in ComponentDist:
    print("Size: %d - Number of Components: %d" % (comp.GetVal1(), comp.GetVal2()))

In [ ]:
get_nodes_and_edges(wccLargestG)

## Q3

In [ ]:
PRankH = snap.TIntFltH()
snap.GetPageRank(G5, PRankH)

In [ ]:
N = 5
top_N = itertools.islice(PRankH, N) # grab the first N elements

for item in top_N:
    print(item, PRankH[item])

In [ ]:
def func_get_top_N(input_hash_table, show_top_N):
    id_value_tuple_list = []
    for item in input_hash_table:
        id_value_tuple_list.append((item, input_hash_table[item]))
        
    sorted_list = sorted(id_value_tuple_list, key=lambda x: x[1], reverse=True)    
    return [i[0] for i in sorted_list[:show_top_N]]

In [ ]:
func_get_top_N(PRankH, 3)

In [ ]:
func_in_out_degree(G5, func_get_top_N(PRankH, 3))

### Q4

In [ ]:
## http://snap.stanford.edu/snappy/doc/reference/GetHits.html?highlight=gethits
NIdHubH = snap.TIntFltH()
NIdAuthH = snap.TIntFltH()
snap.GetHits(G5, NIdHubH, NIdAuthH)

In [ ]:
hub_node_id = func_get_top_N(NIdHubH, 3)
print("Hubs:", hub_node_id)

In [ ]:
authority_node_id = func_get_top_N(NIdAuthH, 3)
print("Authorities:", authority_node_id)

### (optional) Q4

In [ ]:
for curr_id in hub_node_id:
    print("ID:", curr_id)
    func_in_out_degree(G5, [curr_id])

In [ ]:
for curr_id in authority_node_id:
    print("ID:", curr_id)
    func_in_out_degree(G5, [curr_id])

### (optional) Plot

In [ ]:
# import time
# t0 = time.time()
# NIdV = snap.TIntV()
# for i in range(1, 101):
#     NIdV.Add(i)

# demo_graph = snap.GenRndGnm(snap.PNGraph, 200, 1000)
# demo_subgraph = snap.GetSubGraph(demo_graph, NIdV)

# case_name = "plot_subgraph"
# case_txt = case_name+".txt"
# case_png = FOLDER_NAME + case_name+".png"

# snap.DrawGViz(demo_subgraph, snap.gvlDot, case_png, case_name)
# print(f"Elapsed time: {time.time()-t0:.4f} second.")
# Image(case_png)

## Elapsed time: 0.9056 second.

In [ ]:
# t0 = time.time()
# case_name = "plot_graph"
# case_txt = case_name+".txt"
# case_png = case_name+".png"

# snap.DrawGViz(demo_graph, snap.gvlDot, case_png, case_name)
# print(f"Elapsed time: {time.time()-t0:.4f} second.")
# Image(case_png)

## Elapsed time: 727.2719 second.

### (option) Modularity

In [ ]:
def convert_for_modularity(g_in, show_graph=False):
    g_out = snap.ConvertGraph(snap.PUNGraph, g_in)
    CmtyV = snap.TCnComV()
    modularity = snap.CommunityCNM(g_out, CmtyV)
    print(f"The modularity is {modularity:.4f}")
    case_png = 'show_demo.png'
    snap.DrawGViz(g_out, snap.gvlDot, case_png, 'demo')
    display(Image(case_png))
    

In [ ]:
my_graph = snap.GenRndGnm(snap.PNGraph, 6, 10)
convert_for_modularity(my_graph, True)

### (optional) Log-log Plot

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math

def get_xy(graph_in):
    # vector of pairs of integers (size, count)
    CntV = snap.TIntPrV()
    # get degree distribution pairs (degree, count)
    snap.GetOutDegCnt(graph_in, CntV)

    # for item in CntV:
    #     if item.GetVal1() < 10:
    #         print(f"{item.GetVal2()} nodes with out-degree {item.GetVal1()}")

    ## Note:
    ## pair (x, y):  
    ## y is the number of nodes in the network 
    ## with out-degree equal to x.

    pair_list = []
    for item in CntV:
        if item.GetVal1() > 0:
#             print(f" {item.GetVal1():,.0f} out-degree: {item.GetVal2()} nodes")
            pair_list.append((item.GetVal1(), item.GetVal2()))
    
    x_data = [i[0] for i in pair_list]
    y_data = [i[1] for i in pair_list]
    return x_data, y_data

def fit_xy(x_data, y_data):
    log_x = [math.log(i, 10) for i in x_data]
    log_y = [math.log(i, 10) for i in y_data]
    slope, intercept = np.polyfit(log_x, log_y, 1)
    fitted_y = [math.pow(i, slope)* math.pow(10, intercept) for i in x_data]
    return x_data, fitted_y


## Out degree plot
> Each data point is a pair (x, y) where x is a positive integer and y is the number of nodes in the network with <span style="color:yellow">**out-degree** </span> equal to x.

### Stackoverflow

> An edge (a, b) in the network means that person a endorsed an answer from person b on a Java-related question.

In [ ]:
x_, y_ = get_xy(G5)
_, y_fitted_ = fit_xy(x_, y_)

plt.figure(figsize= [8, 6])
plt.loglog(x_, y_)
plt.loglog(x_, y_fitted_)
plt.show()

### Wiki-vote
> An edge (a, b) ∈ E means that user a voted on user b.

In [ ]:
case_name = "wiki-Vote"
case_txt = case_name+".txt"
g_hw0_q2 = snap.LoadEdgeList(snap.PNGraph, case_txt, 0, 1)

x_, y_ = get_xy(g_hw0_q2)
_, y_fitted_ = fit_xy(x_, y_)

plt.figure(figsize= [8, 6])
plt.loglog(x_, y_)
plt.loglog(x_, y_fitted_)
plt.show()

## In degree plot
> Each data point is a pair (x, y) where x is a positive integer and y is the number of nodes in the network with <span style="color:orange">**in-degree** </span> equal to x.

In [ ]:
def get_xy_GetInDegCnt(graph_in, print_values=False):
    # vector of pairs of integers (size, count)
    CntV = snap.TIntPrV()
    # get degree distribution pairs (degree, count)
    snap.GetInDegCnt(graph_in, CntV)

    ## Note:
    ## pair (x, y):  
    ## y is the number of nodes in the network 
    ## with out-degree equal to x.

    pair_list = []
    for item in CntV:
        if item.GetVal1() > 0:
            if print_values:
                print(f" {item.GetVal1():,.0f} in-degree: {item.GetVal2()} nodes")
            pair_list.append((item.GetVal1(), item.GetVal2()))
    
    x_data = [i[0] for i in pair_list]
    y_data = [i[1] for i in pair_list]
    return x_data, y_data

### Stackoverflow

In [ ]:
x_, y_ = get_xy_GetInDegCnt(G5)
_, y_fitted_ = fit_xy(x_, y_)

plt.figure(figsize= [8, 6])
plt.loglog(x_, y_)
plt.loglog(x_, y_fitted_)
plt.show()

### Wiki-vote

In [ ]:
case_name = "wiki-Vote"
case_txt = case_name+".txt"
g_hw0_q2 = snap.LoadEdgeList(snap.PNGraph, case_txt, 0, 1)

x_, y_ = get_xy_GetInDegCnt(g_hw0_q2)
_, y_fitted_ = fit_xy(x_, y_)

plt.figure(figsize= [8, 6])
plt.loglog(x_, y_)
plt.loglog(x_, y_fitted_)
plt.show()

### Thought

In [ ]:
print ("--- Stackoverflow ---")
_ = get_xy_GetInDegCnt(G5, True)

In [ ]:
print ("--- Wiki-vote ---")
_ = get_xy_GetInDegCnt(g_hw0_q2, True)